In [1]:
import torch
print(torch.__version__)


2.1.1+cpu


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.datasets import make_regression
import numpy as np



Original chess state & score

In [15]:


import chess
import chess.engine
import random
import numpy
import math
import chess
import time
import chess.engine
import random

def random_board(max_depth=200):
    while True:
        board = chess.Board()
        depth = random.randrange(0, max_depth)

        for _ in range(depth):
            all_moves = list(board.legal_moves)
            random_move = random.choice(all_moves)
            board.push(random_move)
            if board.is_game_over():
                break

        score = stockfish(board, 5)

        # Check if the score is not None and not nan
        if score is not None and not math.isnan(score):
            return board

# Rest of your code...

def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci(r"C:\Users\shrey\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        
        # Check if the score is not None before accessing the score() method
        score = result['score'].white().score() if result['score'].white() is not None else 0.0

        return float(score) if score is not None else float('nan')


board = random_board()
print(board)
#print(stockfish(board, 10))

squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}
def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
    board3d = numpy.zeros((14,8,8), dtype=numpy.int8)
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = numpy.unravel_index(square, (8,8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = numpy.unravel_index(square, (8,8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    #print(board3d)
    return board3d
#split_dims(board)


r . . . . . . R
. . . . . . k .
p P . P . . . .
. . p . . P p .
. . P . . . . .
. . . . . . . P
R . . . . K . .
N r . . . . . .


Torch CNN(original and modified)

In [13]:


class ChessEvalNet(nn.Module):
    def __init__(self):
        super(ChessEvalNet, self).__init__()
        self.conv1 = nn.Conv2d(14, 32, kernel_size=3, padding=1)  # 14 input channels for the chess board representation
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 128 * 8 * 8)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [19]:
class ChessDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Define hyperparameters
batch_size = 32
num_epochs = 10

# Create dataset and dataloader
dataset = ChessDataset(x, y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class ChessEvalNet(nn.Module):
    def __init__(self):
        super(ChessEvalNet, self).__init__()
        self.conv1 = nn.Conv2d(14, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, optimizer, and loss function
model = ChessEvalNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

NameError: name 'Dataset' is not defined

Data generation

In [16]:
# Number of samples
size_data = 1000


x = np.zeros((size_data, 14, 8, 8), dtype=np.int8)
y = np.zeros(size_data, dtype=np.float32)

start_time = time.time()

for i in range(size_data):
    board = random_board()
    
    x[i] = split_dims(board)
    y[i] = stockfish(board, 5)

    if i % 100 == 0 and i != 0:
        current_time = time.time()
        elapsed_time = current_time - start_time
        print(f"Generated {i} samples, Time taken: {elapsed_time:.2f} seconds")
        start_time = current_time

print("Data generation complete!")

Generated 100 samples, Time taken: 82.44 seconds
Generated 200 samples, Time taken: 90.12 seconds
Generated 300 samples, Time taken: 93.02 seconds
Generated 400 samples, Time taken: 89.92 seconds
Generated 500 samples, Time taken: 80.72 seconds
Generated 600 samples, Time taken: 79.98 seconds
Generated 700 samples, Time taken: 82.76 seconds
Generated 800 samples, Time taken: 86.30 seconds
Generated 900 samples, Time taken: 86.86 seconds
Data generation complete!


save and load data

In [17]:
np.savez('chess_dataset.npz', x=x, y=y)

print("Dataset saved to chess_dataset.npz")

Dataset saved to chess_dataset.npz


In [ ]:
import numpy as np


data = np.load('chess_dataset.npz')
x = data['x']
y = data['y']

print("Data loaded from chess_dataset.npz")
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

data to tensor

In [23]:
from torch.utils.data import DataLoader, TensorDataset
x_tensor = torch.tensor(x, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)


dataset = TensorDataset(x_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [38]:
class ChessEvalNet(nn.Module):
    def __init__(self):
        super(ChessEvalNet, self).__init__()
        self.conv1 = nn.Conv2d(14, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = ChessEvalNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


num_epochs = 50
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/50], Loss: 380045.6875
Epoch [2/50], Loss: 642252.5
Epoch [3/50], Loss: 271466.78125
Epoch [4/50], Loss: 632578.375
Epoch [5/50], Loss: 129062.1328125
Epoch [6/50], Loss: 400927.03125
Epoch [7/50], Loss: 263612.96875
Epoch [8/50], Loss: 184865.34375
Epoch [9/50], Loss: 757465.6875
Epoch [10/50], Loss: 72700.53125
Epoch [11/50], Loss: 177799.671875
Epoch [12/50], Loss: 115838.328125
Epoch [13/50], Loss: 185991.078125
Epoch [14/50], Loss: 326594.96875
Epoch [15/50], Loss: 228458.0
Epoch [16/50], Loss: 479005.4375
Epoch [17/50], Loss: 83889.234375
Epoch [18/50], Loss: 290810.0625
Epoch [19/50], Loss: 167040.90625
Epoch [20/50], Loss: 119600.890625
Epoch [21/50], Loss: 238912.359375
Epoch [22/50], Loss: 167457.71875
Epoch [23/50], Loss: 1136638.875
Epoch [24/50], Loss: 122035.65625
Epoch [25/50], Loss: 115704.1171875
Epoch [26/50], Loss: 48390.65625
Epoch [27/50], Loss: 223062.78125
Epoch [28/50], Loss: 138415.484375
Epoch [29/50], Loss: 106473.1875
Epoch [30/50], Loss: 87428.8203

save and load model

In [41]:

torch.save(model.state_dict(), 'chess_eval_net.pth')
print("Model saved to chess_eval_net.pth")


Model saved to chess_eval_net.pth


In [44]:

model = ChessEvalNet()


model.load_state_dict(torch.load('chess_eval_net.pth'))

model.eval()

print("Model loaded from chess_eval_net.pth")



Model loaded from chess_eval_net.pth


playing

In [46]:
import chess
import chess.engine
import numpy as np
import random
import math
import torch




def random_board(max_depth=200):
    while True:
        board = chess.Board()
        depth = random.randrange(0, max_depth)

        for _ in range(depth):
            all_moves = list(board.legal_moves)
            random_move = random.choice(all_moves)
            board.push(random_move)
            if board.is_game_over():
                break

        score = stockfish(board, 5)

        
        if score is not None and not math.isnan(score):
            return board

def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci(r"C:\Users\shrey\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        
        
        score = result['score'].white().score() if result['score'].white() is not None else 0.0

        return float(score) if score is not None else float('nan')

squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}
def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
    board3d = np.zeros((14,8,8), dtype=np.int8)
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8,8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8,8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    return board3d

def minimax_eval(board):
    board3d = split_dims(board)
    board3d = np.expand_dims(board3d, 0)
    board3d_tensor = torch.tensor(board3d, dtype=torch.float32)
    with torch.no_grad():
        output = model(board3d_tensor)
    return output.item()

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return minimax_eval(board)
    if maximizing_player:
        max_eval = -np.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = np.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def get_ai_move(board, depth):
    max_move = None
    max_eval = -np.inf
    for move in board.legal_moves:
        board.push(move)
        eval = minimax(board, depth - 1, -np.inf, np.inf, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            max_move = move
    return max_move 


board = random_board()
print(board)
move = get_ai_move(board, 3)
print("Best move according to minimax:", move)

. . . . . . . .
. . k . . . . B
. . . p . R r .
P . . . . . P .
. p . . . N . .
. . p K P . . .
. . . . . . . N
. . . . . . . .
Best move according to minimax: f4h5


In [2]:
def get_ai_move(board, depth):
    max_move = None
    max_eval = -np.inf
    
    # Check if the current side to move is white or black
    maximizing_player = board.turn == chess.WHITE
    
    for move in board.legal_moves:
        board.push(move)
        
        # Evaluate the board using the neural network
        eval = minimax_eval(board) if depth <= 1 else minimax(board, depth - 1, -np.inf, np.inf, maximizing_player)
        
        board.pop()
        
        if maximizing_player and eval > max_eval:
            max_eval = eval
            max_move = move
        elif not maximizing_player and eval < max_eval:
            max_eval = eval
            max_move = move
            
    return max_move

def generate_stockfish_move(board, depth):
    max_move = None
    max_eval = -np.inf
    for move in board.legal_moves:
        board.push(move)
        eval = minimax(board, depth - 1, -np.inf, np.inf, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            max_move = move
    return max_move 

# Initialize the board
board = chess.Board()

while True:
    # Neural network's move
    move = get_ai_move(board, 1)
    board.push(move)
    print(f'\nNeural Network\'s move:\n{board}')
    if board.is_game_over():
        break
    
    # Stockfish's move
    stockfish_move = generate_stockfish_move(board, 3)
    board.push(stockfish_move)
    print(f'\nStockfish\'s move:\n{board}')
    if board.is_game_over():
        break

NameError: name 'chess' is not defined